<a href="https://colab.research.google.com/github/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm/blob/oscar1/notebooks/how_to_use_wandb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Wandb Install, Login, Import

In [1]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 35.6 MB/s 
     |████████████████████████████████| 182 kB 59.7 MB/s 
     |████████████████████████████████| 166 kB 66.4 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 166 kB 71.5 MB/s 
     |████████████████████████████████| 162 kB 51.4 MB/s 
     |████████████████████████████████| 162 kB 76.6 MB/s 
     |████████████████████████████████| 158 kB 69.9 MB/s 
     |████████████████████████████████| 157 kB 75.8 MB/s 
     |████████████████████████████████| 157 kB 76.1 MB/s 
     |████████████████████████████████| 157 kB 73.7 MB/s 
     |████████████████████████████████| 157 kB 75.5 MB/s 
     |████████████████████████████████| 157 kB 75.0 MB/s 
     |████████████████████████████████| 157 kB 74.6 MB/s 
     |████████████████████████████████| 157 kB 78.1 MB/s 
     |███████████████████████████

Oscar 2022.11/13(7)_a10.46
```
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
```

In [2]:
!wandb login "6f19b1e6735ebc69af24f18d5b426262416027fb"

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [3]:
import wandb

# Run Model

## Clone team's code

In [4]:
!git clone -b main "https://github.com/ab7289-tandon-nyu/csgy6953_DeepLearning_Midterm.git"

Cloning into 'csgy6953_DeepLearning_Midterm'...
remote: Enumerating objects: 331, done.
remote: Counting objects: 100% (197/197), done.
remote: Compressing objects: 100% (112/112), done.
remote: Total 331 (delta 129), reused 128 (delta 85), pack-reused 134
Receiving objects: 100% (331/331), 98.05 KiB | 446.00 KiB/s, done.
Resolving deltas: 100% (206/206), done.


In [5]:
!cp -r /content/csgy6953_DeepLearning_Midterm/src/ .

## Import, Seed, Device

In [6]:
import torch
import torch.nn as nn

import time
import random

In [7]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Load data

In [8]:
from src.data import get_transformed_data, make_data_loaders
from src.transforms import make_transforms

BATCH_SIZE = 512
VALID_RATIO = 0.1

train_data, valid_data, test_data = \
get_transformed_data(make_transforms=make_transforms, valid_ratio=VALID_RATIO)

train_iter, valid_iter, test_iter = \
make_data_loaders(train_data, valid_data, test_data, BATCH_SIZE)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting .data/cifar-10-python.tar.gz to .data
Files already downloaded and verified
Files already downloaded and verified


## Model

In [9]:
from src.model import ResNet, StemConfig
from src.utils import initialize_parameters, epoch_time

architecture = [
    (1, 64, 0.5),
    (1, 128, 0.5),
    (1, 256, 0.5),
    (1, 512, 0.5),
]

config = StemConfig(num_channels=64, kernel_size=3, stride=1, padding=1)
model  = ResNet(architecture, stem_config=config, output_size=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [10]:
# intialize a new model

inputs = torch.empty((BATCH_SIZE, 3, 32, 32))
inputs.normal_()

model = model.to(device)

outputs = model(inputs.to(device)) 
# (Oscar) observation: 2022.11/13(7)_a08.14: internally, this converts all nn.LazyConv2d layers to nn.Conv2d
# to see this, run print(model) both before and after this operation


print(outputs.size())

torch.Size([512, 10])


In [32]:
# initialize model weights
model.apply(initialize_parameters);

## Count parameters

In [13]:
## utils.py

def count_parameters(model):
    num_parameters = 0
    num_parameters_requiring_grad = 0
    for p in model.parameters():
        numel = p.numel()
        num_parameters += numel
        if p.requires_grad:
            num_parameters_requiring_grad += numel
    return num_parameters, num_parameters_requiring_grad


In [14]:
num_parameters, num_parameters_requiring_grad = count_parameters(model)

assert num_parameters == num_parameters_requiring_grad # our project is using no "frozen parameters"

print(f"num params: {num_parameters:,}")

num params: 4,904,330


## Train, validate, log

In [24]:
from src.engine import train_one_epoch, evaluate

In [25]:
EPOCHS  = 10

# observation 2022.11/13(7)_a10.59:
# each epoch takes 45s to train + 1s to validate.
# so 5 min = 300s / 45s/epoch = 7 epochs

learning_rate = 1e-3

In [26]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [27]:
## setup wandb logging

assert 'wandb' in locals() # 'wandb' has been imported already

ENTITY_NAME  = 'dlf22_mini_project' # this is our team name
PROJECT_NAME = 'Junk_Test_Project'
RUN_NAME     = 'resnet_oscar_test_run_2022.11.13.a11.00'

WANDB_CONFIG = \
{"architecture"  : architecture,    # the model
 "num_parameters": num_parameters,  # the model
 "learning_rate" : learning_rate,   # how to gradient descent
 "batch_size"    : BATCH_SIZE,      # how to gradient descent
"epochs"         : EPOCHS,}         # train for how long

wandb\
.init(name=RUN_NAME, project=PROJECT_NAME, entity=ENTITY_NAME,config=WANDB_CONFIG)

epoch,▁▁▅▅██
test_acc,▁
test_loss,▁
train_acc,▁▆█
train_loss,█▃▁
val_acc,▁▇█
val_loss,█▁▁
epoch,3
test_acc,0.59856
test_loss,1.18943
train_acc,0.59153


In [28]:
# save to disk the "best" model === the model with the lowest validation loss
best_model_path = RUN_NAME + '.pt'

In [29]:
best_loss = float('inf')
for epoch in range(1, EPOCHS+1):

    print(f"Epoch {epoch}")
    
    ## TRAIN
    start = time.time()
    # train
    train_loss, train_acc  = train_one_epoch(model, train_iter, criterion, optimizer, device)
    # log & echo
    train_mins, train_secs = epoch_time(start, time.time())
    wandb.log({"train_loss": train_loss,
               "train_acc" : train_acc,
               "epoch"     : epoch})
    print(f"\tTrain elapsed: {train_mins}:{train_secs}, loss: {train_loss:.4f}, acc: {train_acc * 100:.2f}%")

    ## VALIDATE
    start = time.time()
    # validate
    val_loss, val_acc  = evaluate(model, valid_iter, criterion, device)
    # log & echo   
    val_mins, val_secs = epoch_time(start, time.time())
    wandb.log({"val_loss": val_loss,
               "val_acc" : val_acc,
               "epoch"   : epoch,})
    print(f"\tValidation elapsed: {val_mins}:{val_secs}, loss: {val_loss:.4f}, acc: {val_acc * 100:.2f}%")

    ## Memorize "best" model === the model with the lowest validation loss
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), best_model_path)

Epoch 1
	Train elapsed: 0:45, loss: 1.0494, acc: 62.55%
	Validation elapsed: 0:1, loss: 0.9813, acc: 66.66%
Epoch 2
	Train elapsed: 0:44, loss: 0.9272, acc: 66.89%
	Validation elapsed: 0:1, loss: 0.9695, acc: 66.55%
Epoch 3
	Train elapsed: 0:45, loss: 0.8484, acc: 69.89%
	Validation elapsed: 0:1, loss: 0.8616, acc: 70.73%
Epoch 4
	Train elapsed: 0:45, loss: 0.7846, acc: 72.10%
	Validation elapsed: 0:1, loss: 0.7193, acc: 75.38%
Epoch 5
	Train elapsed: 0:45, loss: 0.7261, acc: 74.38%
	Validation elapsed: 0:1, loss: 0.6934, acc: 75.42%
Epoch 6
	Train elapsed: 0:46, loss: 0.6779, acc: 76.34%
	Validation elapsed: 0:1, loss: 0.6201, acc: 78.18%
Epoch 7
	Train elapsed: 0:46, loss: 0.6316, acc: 77.79%
	Validation elapsed: 0:1, loss: 0.5876, acc: 79.53%
Epoch 8
	Train elapsed: 0:46, loss: 0.5989, acc: 79.00%
	Validation elapsed: 0:1, loss: 0.5728, acc: 80.79%
Epoch 9
	Train elapsed: 0:46, loss: 0.5618, acc: 80.30%
	Validation elapsed: 0:1, loss: 0.5362, acc: 81.70%
Epoch 10
	Train elapsed: 0:4

## Test, log

In [30]:
model.load_state_dict(torch.load(best_model_path))

<All keys matched successfully>

In [31]:
## TEST
# test
test_loss, test_acc = evaluate(model.to(device), test_iter, criterion, device)
# log & echo
wandb.log({"test_loss": test_loss,
           "test_acc" : test_acc,})
print(f"Test Loss: {test_loss:.4f}\nTest Accuracy: {test_acc * 100:.2f}%")

Test Loss: 0.5478
Test Accuracy: 81.08%
